In [1]:
# ================================================
# ✅ 1️⃣ LIBRARIES & SETUP
# ================================================
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import torchvision.transforms as transforms
from torchvision import models

# ================================================
# ✅ 2️⃣ PATHS
# ================================================
image_dir = "/kaggle/input/basem/images"
input_csv = "/kaggle/input/basem/dataset.csv"

# ================================================
# ✅ 3️⃣ LOAD & PREPROCESS CSV
# ================================================
df = pd.read_csv(input_csv)

existing_data = []
for _, row in df.iterrows():
    image_filename = row['image_path']
    full_image_path = os.path.join(image_dir, image_filename)
    if os.path.exists(full_image_path):
        label_converted = row['label 2'] - 1
        existing_data.append({
            'Image_path': full_image_path,
            'Label_Sentiment': label_converted
        })

processed_df = pd.DataFrame(existing_data)

# ================================================
# ✅ 4️⃣ DATA SPLITS
# ================================================
train_df, temp_df = train_test_split(processed_df, test_size=0.3, stratify=processed_df['Label_Sentiment'], random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=1/3, stratify=temp_df['Label_Sentiment'], random_state=42)

for df_name, df_ in [('train', train_df), ('test', test_df), ('val', val_df)]:
    df_['label'] = df_['Label_Sentiment']
    df_.to_csv(f'/kaggle/working/{df_name}_vision_only.csv', index=False)

# ================================================
# ✅ 5️⃣ DEVICE & TRANSFORMS
# ================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation/test
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ================================================
# ✅ 6️⃣ VISION-ONLY DATASET
# ================================================
class VisionOnlyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['Image_path']).convert('RGB')
        label = row['label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# ================================================
# ✅ 7️⃣ DATALOADERS
# ================================================
batch_size = 16

train_dataset = VisionOnlyDataset(train_df, transform=train_transform)
val_dataset = VisionOnlyDataset(val_df, transform=val_test_transform)
test_dataset = VisionOnlyDataset(test_df, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ================================================
# ✅ 8️⃣ DENSENET161 MODEL
# ================================================
class DenseNet161Classifier(nn.Module):
    def __init__(self, num_classes=3, pretrained=True):
        super(DenseNet161Classifier, self).__init__()
        self.densenet = models.densenet161(pretrained=pretrained)
        
        # Replace the classifier layer
        num_features = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.densenet(x)

# ================================================
# ✅ 9️⃣ MODEL INITIALIZATION
# ================================================
model = DenseNet161Classifier(num_classes=3, pretrained=True).to(device)

# ================================================
# ✅ 🔟 LOSS & OPTIMIZER
# ================================================
# Calculate class weights for balanced training
class_weights = train_df['label'].value_counts().sort_index().tolist()
total = sum(class_weights)
weights = [total / c for c in class_weights]
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(weights).to(device))

# Optimizer with different learning rates for feature extractor and classifier
optimizer = AdamW([
    {'params': model.densenet.features.parameters(), 'lr': 1e-5},  # Lower LR for pretrained features
    {'params': model.densenet.classifier.parameters(), 'lr': 1e-4}  # Higher LR for new classifier
], weight_decay=1e-4)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# ================================================
# ✅ 1️⃣1️⃣ TRAINING LOOP
# ================================================
num_epochs = 25
patience = 5
patience_counter = 0
best_val_loss = float('inf')

print(f"Training on {len(train_dataset)} samples")
print(f"Validating on {len(val_dataset)} samples")
print(f"Testing on {len(test_dataset)} samples")

for epoch in range(num_epochs):
    # ============================================================
    # TRAINING PHASE
    # ============================================================
    model.train()
    total_train_loss = 0
    train_predictions = []
    train_labels = []

    for images, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        
        # Store predictions for training accuracy
        predictions = torch.argmax(outputs, dim=1)
        train_predictions.extend(predictions.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = accuracy_score(train_labels, train_predictions)

    # ============================================================
    # VALIDATION PHASE
    # ============================================================
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_val_loss += loss.item()
            
            # Store predictions for metrics
            predictions = torch.argmax(outputs, dim=1)
            val_predictions.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_predictions)
    
    # Update learning rate
    scheduler.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
    print("-" * 50)

    # ============================================================
    # EARLY STOPPING CHECK
    # ============================================================
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_densenet161_model.pt")
        print("✅ Validation loss improved — model saved.")
    else:
        patience_counter += 1
        print(f"⏰ No improvement — patience {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break

# ================================================
# ✅ 1️⃣2️⃣ FINAL TEST EVALUATION
# ================================================
print("\n🔍 Loading best model for final evaluation...")
model.load_state_dict(torch.load("best_densenet161_model.pt"))
model.eval()

test_predictions = []
test_labels = []
total_test_loss = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Final Test Evaluation"):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        total_test_loss += loss.item()
        
        predictions = torch.argmax(outputs, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Calculate final metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='weighted')
cm = confusion_matrix(test_labels, test_predictions)

print("\n📊 FINAL TEST RESULTS (Vision-Only DenseNet161):")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test Loss: {total_test_loss/len(test_loader):.4f}")
print(f"\nConfusion Matrix:\n{cm}")

# ================================================
# ✅ 1️⃣3️⃣ CLASS-WISE METRICS
# ================================================
print("\n📋 CLASS-WISE METRICS:")
precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
    test_labels, test_predictions, average=None
)

class_names = ['Negative', 'Neutral', 'Positive']
for i, class_name in enumerate(class_names):
    print(f"{class_name}: Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, F1={f1_per_class[i]:.4f}, Support={support[i]}")

print(f"\nDataset Distribution:")
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")
print(f"Class distribution in training set:")
print(train_df['label'].value_counts().sort_index())

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 158MB/s] 


Training on 3156 samples
Validating on 451 samples
Testing on 902 samples


Validation Epoch 1: 100%|██████████| 29/29 [00:07<00:00,  3.94it/s]


Epoch [1/25]
Train Loss: 1.0386 | Train Acc: 0.4743
Val Loss: 0.8988 | Val Acc: 0.6341
Learning Rate: 0.000010
--------------------------------------------------
✅ Validation loss improved — model saved.


Validation Epoch 2: 100%|██████████| 29/29 [00:06<00:00,  4.25it/s]


...

Epoch [7/25]
Train Loss: 0.7027 | Train Acc: 0.6844
Val Loss: 0.9169 | Val Acc: 0.6253
Learning Rate: 0.000001
--------------------------------------------------
⏰ No improvement — patience 4/5


Validation Epoch 8: 100%|██████████| 29/29 [00:06<00:00,  4.57it/s]


Epoch [8/25]
Train Loss: 0.6355 | Train Acc: 0.7148
Val Loss: 0.9284 | Val Acc: 0.6120
Learning Rate: 0.000001
--------------------------------------------------
⏰ No improvement — patience 5/5
🛑 Early stopping triggered at epoch 8

🔍 Loading best model for final evaluation...


Final Test Evaluation: 100%|██████████| 57/57 [00:14<00:00,  4.03it/s]


📊 FINAL TEST RESULTS (Vision-Only DenseNet161):
Test Accuracy: 0.6319
Test Precision: 0.6488
Test Recall: 0.6319
Test F1-Score: 0.6368
Test Loss: 0.8193

Confusion Matrix:
[[257  96  49]
 [ 72 224  57]
 [ 15  43  89]]

📋 CLASS-WISE METRICS:
Negative: Precision=0.7471, Recall=0.6393, F1=0.6890, Support=402
Neutral: Precision=0.6171, Recall=0.6346, F1=0.6257, Support=353
Positive: Precision=0.4564, Recall=0.6054, F1=0.5205, Support=147

Dataset Distribution:
Training samples: 3156
Validation samples: 451
Test samples: 902
Class distribution in training set:
label
0    1404
1    1237
2     515
Name: count, dtype: int64
